## Replicate the baseline data processing performed in MTS_Analysis R code

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

In [3]:
# python code that performs the following R code
"""
print("Loading the data")
GE <- data.table::fread(paste0(expression_dir, "/ge.csv")) %>%  # gene exp.
  as.matrix(., rownames = "V1")
CNA <- data.table::fread(paste0(expression_dir, "/cna.csv")) %>%  # copy num
  as.matrix(., rownames = "V1")
MET <- data.table::fread(paste0(expression_dir, "/met.csv")) %>%  # metablomics
  as.matrix(., rownames = "V1")
miRNA <- data.table::fread(paste0(expression_dir, "/mirna.csv")) %>%  # micro RNA
  as.matrix(., rownames = "V1")
MUT <- data.table::fread(paste0(expression_dir, "/mut.csv")) %>%  # mutations
  as.matrix(., rownames = "V1")
PROT <- data.table::fread(paste0(expression_dir, "/prot.csv")) %>%  # proteomics
  as.matrix(., rownames = "V1")
XPR <- data.table::fread(paste0(expression_dir, "/xpr.csv")) %>%  # CRISPR KO
  as.matrix(., rownames = "V1")
LIN <- data.table::fread(paste0(expression_dir, "/lin.csv")) %>%  # lineage
  as.matrix(., rownames = "V1")
shRNA <- data.table::fread(paste0(expression_dir, "/shrna.csv")) %>%  # lineage
  as.matrix(., rownames = "V1")
repurposing <- data.table::fread(paste0(expression_dir, "/rep.csv")) %>%  # repurposing
  as.matrix(., rownames = "V1")
repurposing_meta <- data.table::fread(paste0(expression_dir, "/rep_info.csv")) %>%
  dplyr::mutate(column_name = paste("REP", column_name, sep = "_"),
                name = stringr::str_replace_all(name, "[[:punct:]\\s]+", "-")) %>%
  dplyr::select(-dose, -screen_id)

continuous_features <- list(GE, CNA, MET, miRNA, PROT, XPR, shRNA, repurposing)
continuous_names <- c("GE", "CNA", "MET", "miRNA", "PROT", "XPR", "shRNA", "REP")

discrete_features <- list(LIN, MUT)
discrete_names <- c("LIN", "MUT")
"""

# Load the data from the R code above
feature_dir = "/mnt/c/Users/nick/Desktop/courses/rotations/corsello_lab/data/depmap_public-22q1-305b_v24"
GE = pd.read_csv(f"{feature_dir}/ge.csv", index_col=0)
CNA = pd.read_csv(f"{feature_dir}/cna.csv", index_col=0)
MET = pd.read_csv(f"{feature_dir}/met.csv", index_col=0)
miRNA = pd.read_csv(f"{feature_dir}/mirna.csv", index_col=0)
MUT = pd.read_csv(f"{feature_dir}/mut.csv", index_col=0)
PROT = pd.read_csv(f"{feature_dir}/prot.csv", index_col=0)
XPR = pd.read_csv(f"{feature_dir}/xpr.csv", index_col=0)
LIN = pd.read_csv(f"{feature_dir}/lin.csv", index_col=0)
shRNA = pd.read_csv(f"{feature_dir}/shrna.csv", index_col=0)
repurposing = pd.read_csv(f"{feature_dir}/rep.csv", index_col=0)
repurposing_meta = pd.read_csv(f"{feature_dir}/rep_info.csv")
# Add new columns
repurposing_meta['column_name'] = 'REP_' + repurposing_meta['column_name'].astype(str)
repurposing_meta['name'] = repurposing_meta['name'].astype(str).apply(lambda x: re.sub(r'[^\w\s]', '-', x))
# Remove columns
repurposing_meta = repurposing_meta.drop(columns=['dose', 'screen_id'])

# Create a list of the continuous features
continuous_features = [GE, CNA, MET, miRNA, PROT, XPR, shRNA, repurposing]
continuous_names = ["GE", "CNA", "MET", "miRNA", "PROT", "XPR", "shRNA", "REP"]

# Create a list of the discrete features
discrete_features = [LIN, MUT]
discrete_names = ["LIN", "MUT"]

In [36]:
# Python code that performs the following R code
"""
# combinations of features for multivariate
X.all <- data.table::fread(paste0(expression_dir, "/x-all.csv")) %>%
  subset(select=which(!duplicated(names(.)))) %>%
  unique() %>%
  as.matrix(., rownames = "V1")
X.ccle <- data.table::fread(paste0(expression_dir, "/x-ccle.csv")) %>%
  subset(select=which(!duplicated(names(.)))) %>%
  unique() %>%
  as.matrix(., rownames = "V1")
"""

# Load the data from the R code above
X_all = pd.read_csv(f"{feature_dir}/x-all.csv")
X_all = X_all.loc[:,~X_all.columns.duplicated()]
X_all = X_all.drop_duplicates()
X_all = X_all.set_index(X_all.columns[0])
X_all.index.name = "ccle_name"

X_ccle = pd.read_csv(f"{feature_dir}/x-ccle.csv")
X_ccle = X_ccle.loc[:,~X_ccle.columns.duplicated()]
X_ccle = X_ccle.drop_duplicates()
X_ccle = X_ccle.set_index(X_ccle.columns[0])
X_ccle.index.name = "ccle_name"

In [6]:
# Python code that performs the following R code
"""
LFC <- data.table::fread(paste0(response_dir, "/amg-232_v2.csv")) %>%
  dplyr::distinct(pert_name, ccle_name, culture, pert_idose, pert_mfc_id, LFC.cb) %>%
  ##cb : combat (algorithm for batch normalization)
  ###GET THESE COLUMNS(pert_name, culture, pert_idose, preft_mfc_id) FROM TREATMENT METADATA
  dplyr::rename(response = LFC.cb, dose = pert_idose) %>%
  dplyr::filter(is.finite(response))
"""
# get the response_dir
response_dir = "/mnt/c/Users/nick/Desktop/courses/rotations/corsello_lab/data/responsedir"

# Load the data from the R code above
LFC = pd.read_csv(f"{response_dir}/amg-232_v2.csv")
LFC = LFC.drop_duplicates(subset=['pert_name', 'ccle_name', 'culture', 'pert_idose', 'pert_mfc_id', 'LFC.cb'])
LFC = LFC.rename(columns={'LFC.cb': 'response', 'pert_idose': 'dose'})
LFC = LFC[LFC['response'].notna()]

In [7]:
LFC

,response,ccle_name,pert_name,culture,dose,pert_dose_unit,pert_mfc_id
0,-1.185322,KYSE510_OESOPHAGUS,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
1,-1.147726,HEC1A_ENDOMETRIUM,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
2,-0.048768,MIAPACA2_PANCREAS,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
3,-0.789618,SW620_LARGE_INTESTINE,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
4,-3.910685,SKHEP1_LIVER,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
...,...,...,...,...,...,...,...
539,-0.984816,KPNYN_AUTONOMIC_GANGLIA,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
541,-1.686533,MHHNB11_AUTONOMIC_GANGLIA,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
542,-0.933229,SKNDZ_AUTONOMIC_GANGLIA,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8
543,-1.450440,SKNSH_AUTONOMIC_GANGLIA,AMG-232,PR500,2.5,uM,BRD-K64925568-001-01-8


In [9]:
# python code that performs the following R code
"""
# get principle components of lineage (for confounders)
LIN_PCs <- gmodels::fast.prcomp(LIN);
# only get PCs where sdev is greater than 0.2
LIN_PCs <-  LIN %*% LIN_PCs$rotation[, LIN_PCs$sdev  > 0.2]
"""
from sklearn.decomposition import PCA

# Perform PCA on LIN
pca = PCA()
LIN_PCs = pca.fit_transform(LIN)

# Get the PCs with std greater than 0.2
selected_PCs = pca.components_[pca.explained_variance_ > 0.2]
LIN_PCs = LIN @ selected_PCs.T

In [17]:
# get distinct runs
runs = LFC[['pert_name', 'dose', 'pert_mfc_id']].drop_duplicates()

In [18]:
runs

,pert_name,dose,pert_mfc_id
0,AMG-232,2.5,BRD-K64925568-001-01-8


In [19]:
continuous = {}
discrete = {}
rf_results = {}
multi_models = {}

In [20]:
pert_name = runs['pert_name'].iloc[0]
pert_mfc_id = runs['pert_mfc_id'].iloc[0]
dose = runs['dose'].iloc[0]

In [33]:
# get the relevant responses
Y = LFC.loc[
    (LFC.pert_name == pert_name) & (LFC.pert_mfc_id == pert_mfc_id) & (LFC.dose == dose),
    :
]
y = Y.loc[:, ['ccle_name', 'response']].set_index('ccle_name')

In [34]:
y

,response
ccle_name,
KYSE510_OESOPHAGUS,-1.185322
HEC1A_ENDOMETRIUM,-1.147726
MIAPACA2_PANCREAS,-0.048768
SW620_LARGE_INTESTINE,-0.789618
SKHEP1_LIVER,-3.910685
...,...
KPNYN_AUTONOMIC_GANGLIA,-0.984816
MHHNB11_AUTONOMIC_GANGLIA,-1.686533
SKNDZ_AUTONOMIC_GANGLIA,-0.933229


In [35]:
# multivariate analysis


,GE_TSPAN6,GE_TNMD,GE_DPM1,GE_SCYL3,GE_C1orf112,GE_FGR,GE_CFH,GE_FUCA2,GE_GCLC,GE_NFYA,...,MET_C56:8 TAG,MET_C56:7 TAG,MET_C56:6 TAG,MET_C56:5 TAG,MET_C56:4 TAG,MET_C56:3 TAG,MET_C56:2 TAG,MET_C58:8 TAG,MET_C58:7 TAG,MET_C58:6 TAG
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
DEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.097611,0.0,5.919102,3.983678,3.733354,0.028569,6.111240,2.963474,3.415488,4.820690,...,6.212027,6.634990,6.509410,6.406298,5.994251,6.008896,5.983564,6.515810,6.153836,6.391265
SNU1196_BILIARY_TRACT,4.712596,0.0,6.406333,2.247928,3.032101,0.028569,0.097611,5.528571,6.383704,3.973611,...,5.894121,6.447221,5.625546,5.692822,5.660319,6.289528,6.055030,6.060976,6.053000,5.520285
ABC1_LUNG,5.236493,0.0,7.005849,2.829850,4.666757,0.014355,0.823749,6.607478,3.975447,4.362470,...,5.983268,6.038920,6.088791,5.806712,5.485296,5.721401,5.896228,5.976406,5.871337,6.045364
KE97_STOMACH,0.070389,0.0,6.649184,2.298658,3.971773,5.852748,0.275007,3.612352,4.145677,3.955127,...,5.936662,6.060182,6.226179,6.384805,6.429166,6.422736,6.179932,6.196477,6.601452,6.649329
YKG1_CENTRAL_NERVOUS_SYSTEM,5.914086,0.0,6.749668,2.809414,4.175525,0.176323,5.859224,6.535275,4.598127,5.063071,...,5.854716,5.240681,5.320836,5.068294,5.145107,5.079931,5.267301,5.147127,4.708627,4.951173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCIH2023_LUNG,4.046142,0.0,5.786335,2.073820,3.909773,0.000000,6.221297,6.615299,7.491372,3.853996,...,5.663912,5.668636,5.589633,5.304429,5.622208,5.671981,5.940422,5.480966,5.455681,5.194612
NB4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.150560,0.0,6.493455,2.097611,3.392317,1.137504,0.137504,5.246028,4.169925,4.455492,...,6.089156,6.141175,6.122040,6.126260,6.008872,6.112959,6.414226,6.300899,6.136446,6.225555
COLO792_SKIN,4.269781,0.0,5.429281,1.757023,3.347666,0.042644,0.443607,5.082362,3.613532,4.185074,...,5.325039,5.766816,5.700176,5.430885,5.704363,5.772005,5.694848,5.439704,5.489285,5.133185


In [37]:
# join features
feat_all = y.merge(X_all, how="left", left_index=True, right_index=True)
feat_ccle = y.merge(X_ccle, how="left", left_index=True, right_index=True)

In [41]:
feat_all

,response,GE_TSPAN6,GE_TNMD,GE_DPM1,GE_SCYL3,GE_C1orf112,GE_FGR,GE_CFH,GE_FUCA2,GE_GCLC,...,MET_C56:8 TAG,MET_C56:7 TAG,MET_C56:6 TAG,MET_C56:5 TAG,MET_C56:4 TAG,MET_C56:3 TAG,MET_C56:2 TAG,MET_C58:8 TAG,MET_C58:7 TAG,MET_C58:6 TAG
ccle_name,,,,,,,,,,,,,,,,,,,,,
KYSE510_OESOPHAGUS,-1.185322,4.968552,0.000000,6.730232,2.400538,4.071248,0.000000,3.270529,6.299208,6.701965,...,5.903735,5.466582,5.125829,5.412568,6.191993,6.444636,6.305175,5.243109,5.081646,5.356238
HEC1A_ENDOMETRIUM,-1.147726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MIAPACA2_PANCREAS,-0.048768,0.097611,0.000000,6.579391,1.769772,3.681449,0.028569,0.176323,6.074891,3.443607,...,5.699993,5.309184,5.335225,5.184828,5.367463,5.865795,5.664049,5.274246,5.126197,5.066231
SW620_LARGE_INTESTINE,-0.789618,5.806066,0.000000,7.007532,1.786596,3.519793,0.028569,0.042644,5.902074,5.172728,...,5.495717,5.346540,5.487956,5.366878,5.594239,6.063419,6.137869,5.214890,5.047763,5.239845
SKHEP1_LIVER,-3.910685,4.395748,0.000000,5.988685,1.655352,3.148934,0.056584,1.238787,5.348374,3.560715,...,5.632193,5.671673,5.535060,5.345089,5.320982,5.451577,6.057451,5.370904,5.281377,5.222446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KPNYN_AUTONOMIC_GANGLIA,-0.984816,4.516015,0.000000,5.645875,2.260026,3.662205,0.000000,0.070389,0.565597,5.011675,...,5.805825,6.184845,6.192903,6.445593,6.166878,5.900550,5.505959,6.227312,6.047117,6.402715
MHHNB11_AUTONOMIC_GANGLIA,-1.686533,2.969012,0.000000,4.987321,2.319040,2.744161,0.000000,0.097611,3.653060,4.464668,...,5.821846,6.475812,6.148949,6.509765,6.672206,6.289194,6.446885,5.965590,6.263530,6.396592
SKNDZ_AUTONOMIC_GANGLIA,-0.933229,0.238787,0.070389,6.987548,2.639232,4.442943,0.000000,0.275007,0.250962,5.085765,...,5.987425,6.029615,6.396747,6.486425,6.750343,6.340906,6.053562,6.597543,6.497785,6.432953


In [42]:
# only get rows with no missing data
feat_all = feat_all.dropna()
feat_ccle = feat_ccle.dropna()

In [45]:
# Train a random forest model 
from sklearn.ensemble import RandomForestRegressor

,response,GE_TSPAN6,GE_TNMD,GE_DPM1,GE_SCYL3,GE_C1orf112,GE_FGR,GE_CFH,GE_FUCA2,GE_GCLC,...,MET_C56:8 TAG,MET_C56:7 TAG,MET_C56:6 TAG,MET_C56:5 TAG,MET_C56:4 TAG,MET_C56:3 TAG,MET_C56:2 TAG,MET_C58:8 TAG,MET_C58:7 TAG,MET_C58:6 TAG
ccle_name,,,,,,,,,,,,,,,,,,,,,
KYSE510_OESOPHAGUS,-1.185322,4.968552,0.000000,6.730232,2.400538,4.071248,0.000000,3.270529,6.299208,6.701965,...,5.903735,5.466582,5.125829,5.412568,6.191993,6.444636,6.305175,5.243109,5.081646,5.356238
MIAPACA2_PANCREAS,-0.048768,0.097611,0.000000,6.579391,1.769772,3.681449,0.028569,0.176323,6.074891,3.443607,...,5.699993,5.309184,5.335225,5.184828,5.367463,5.865795,5.664049,5.274246,5.126197,5.066231
SW620_LARGE_INTESTINE,-0.789618,5.806066,0.000000,7.007532,1.786596,3.519793,0.028569,0.042644,5.902074,5.172728,...,5.495717,5.346540,5.487956,5.366878,5.594239,6.063419,6.137869,5.214890,5.047763,5.239845
SKHEP1_LIVER,-3.910685,4.395748,0.000000,5.988685,1.655352,3.148934,0.056584,1.238787,5.348374,3.560715,...,5.632193,5.671673,5.535060,5.345089,5.320982,5.451577,6.057451,5.370904,5.281377,5.222446
YKG1_CENTRAL_NERVOUS_SYSTEM,-1.013539,5.914086,0.000000,6.749668,2.809414,4.175525,0.176323,5.859224,6.535275,4.598127,...,5.854716,5.240681,5.320836,5.068294,5.145107,5.079931,5.267301,5.147127,4.708627,4.951173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SW579_THYROID,-0.669221,4.288359,0.000000,6.548745,1.906891,3.629939,0.070389,4.564378,6.201242,4.742006,...,5.621257,5.974895,5.825681,5.716037,6.095884,5.920705,5.914835,5.809381,5.739446,5.763773
CHP212_AUTONOMIC_GANGLIA,-1.099385,4.925050,0.000000,6.106223,3.521051,4.552131,0.000000,3.719183,5.053111,5.000000,...,6.083524,5.302452,5.623373,5.319954,5.772304,5.781201,5.922409,5.174163,4.975156,5.295141
KPNYN_AUTONOMIC_GANGLIA,-0.984816,4.516015,0.000000,5.645875,2.260026,3.662205,0.000000,0.070389,0.565597,5.011675,...,5.805825,6.184845,6.192903,6.445593,6.166878,5.900550,5.505959,6.227312,6.047117,6.402715


In [47]:
# cache feature dataframes to disk
cache_dir = "./cache"
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)
feat_all.to_csv(f"{cache_dir}/feat_all.csv")
feat_ccle.to_csv(f"{cache_dir}/feat_ccle.csv")

## Train a random forest model

In [49]:
# import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

In [50]:
# train a random forest model to predict the response from the features 
# response is named "response"
# features are all other columns
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0, verbose=1)
rf.fit(feat_all.drop(columns=['response']), feat_all['response'])

# get the feature importances
importances = pd.DataFrame(rf.feature_importances_, index=feat_all.drop(columns=['response']).columns, columns=['importance'])
importances = importances.sort_values(by='importance', ascending=False)

KeyboardInterrupt: 